In [1]:
import requests
import pandas as pd
from io import StringIO

In [2]:
# -------------------------------------------------------
# 0) Proxy settings (from your FoxyProxy config)
# -------------------------------------------------------
proxies = {
    "http":  "http://localhost:3128",
    "https": "http://localhost:3128",
}

session = requests.Session()
session.proxies.update(proxies)

In [3]:
# -------------------------------------------------------
# 1) Fetch the trigger table for a single run
# -------------------------------------------------------
def fetch_trigger_table(run):
    """
    Download the trigger-details HTML for a given run
    and return the trigger table as a pandas DataFrame.
    """
    url = f"http://www.sphenix-intra.bnl.gov:7815/cgi-bin/trigger_details.py?run={run}"
    r = session.get(url)
    r.raise_for_status()

    tables = pd.read_html(StringIO(r.text))

    for df in tables:
        if "Name" in df.columns and "Live" in df.columns:
            df["Name"] = df["Name"].astype(str).str.strip()
            return df


In [4]:
# -------------------------------------------------------
# 2) Extract Clock and MBD N&S >= 1 live counts
# -------------------------------------------------------
clock_trigger_rate = 9.4 #MHz
def get_live_counts_for_run(run):
    """
    Return (clock_live, mbd_ns_ge1_live, ratio) for a run.
    ratio = mbd_ns_ge1_live / clock_live
    """
    df = fetch_trigger_table(run)

    # Clock trigger
    clock_live_series = df.loc[df["Name"] == "Clock", "Live"]
    if clock_live_series.empty:
        raise RuntimeError(f"'Clock' trigger not found for run {run}")
    clock_live = int(clock_live_series.iloc[0])

    # MBD N&S >= 1
    mbd_series = df.loc[df["Name"].str.contains("MBD N&S >= 1", regex=False), "Live"]
    if mbd_series.empty:
        raise RuntimeError(f"'MBD N&S >= 1' trigger not found for run {run}")
    mbd_live = int(mbd_series.iloc[0])

    ratio = clock_trigger_rate * mbd_live / clock_live if clock_live > 0 else float("nan")
    return clock_live, mbd_live, ratio

In [5]:
# -------------------------------------------------------
# 3) Helper for a list of runs
# -------------------------------------------------------
def summarize_runs(run_list):
    """
    For each run in run_list, compute live counts and ratio.
    Returns a pandas DataFrame.
    """
    rows = []
    for run in run_list:
        try:
            clock_live, mbd_live, ratio = get_live_counts_for_run(run)
            rows.append({
                "run": run,
                "clock_live": clock_live,
                "mbd_ns_ge1_live": mbd_live,
                "mbd_trigger_rate": ratio,
            })
        except Exception as e:
            rows.append({
                "run": run,
                "clock_live": None,
                "mbd_ns_ge1_live": None,
                "mbd_trigger_rate": None,
                "error": str(e),
            })
    return pd.DataFrame(rows)

In [6]:
# -------------------------------------------------------
# 4) Examples
# -------------------------------------------------------

# Single run
run = 48073
clock_live, mbd_live, ratio = get_live_counts_for_run(run)
print(f"Run {run}:")
print(f"  Clock live        = {clock_live}")
print(f"  MBD N&S >= 1 live = {mbd_live}")
print(f"  Trigger Rate = {ratio}")

# Multiple runs
runs = [48073, 48084, 48089]   # replace with your run list
df_runs = summarize_runs(runs)
df_runs


Run 48073:
  Clock live        = 15615217325
  MBD N&S >= 1 live = 2057580963
  Trigger Rate = 1.238616194040066


,run,clock_live,mbd_ns_ge1_live,mbd_trigger_rate
0,48073,15615217325,2057580963,1.238616
1,48084,32425869943,3389250548,0.982517
2,48089,32243656579,3831487599,1.116994


In [7]:
with open("/home/yoren/yumvd.Yandex.Disk/Yura/sPHENIX/emcal/ppRun2024_MB_triggered_sofar.txt") as f:
    runs = [int(line.strip()) for line in f if line.strip()]

df_runs = summarize_runs(runs)
df_runs.to_csv("output/trigger_live_counts.csv", index=False)
df_runs


,run,clock_live,mbd_ns_ge1_live,mbd_trigger_rate
0,47289,6354008810,793594834,1.174029
1,47293,5028686969,602982581,1.127140
2,47306,5738634927,646765450,1.059415
3,47310,16680186659,1791481691,1.009577
4,47313,32302554579,3166756918,0.921522
...,...,...,...,...
945,53856,8983695328,384704022,0.402531
946,53860,33282689317,1305508577,0.368714
947,53861,19348850136,730321299,0.354802
948,53862,33259393207,1202204192,0.339775
